In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import numpy as np
import networkx as nx
import pylab as plt

eventsWorld_Cup = pd.read_json('/content/drive/My Drive/bachelor data/events/events_World_Cup.json')
matchesWorld_Cup = pd.read_json("/content/drive/My Drive/bachelor data/matches/matches_World_Cup.json")
tagsName = pd.read_csv("/content/drive/My Drive/bachelor data/tags2name.csv")
with open('/content/drive/My Drive/bachelor data/events/events_World_Cup.json') as f:
    d = json.load(f)
tags = json_normalize(data=d, record_path='tags', meta=['id'], record_prefix='_')
teams = pd.read_json("/content/drive/My Drive/bachelor data/teams.json")
eventsWorld_CupTags = pd.merge(eventsWorld_Cup,
                       tags,
                       on='id')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


In [ ]:
def checkZones2(dataframe, columnName):
  resultSeries = pd.Series([])
  i = -1
  for index, row in dataframe.iterrows():
    x = (row[columnName])['x']
    y = (row[columnName])['y']
    index = i + 1
    resultSeries = resultSeries.append(pd.Series([int(((x // 33.3) * 5) + 1 + (y // 20))]), ignore_index=True)
  return resultSeries

In [ ]:
# Filling 2D matrix of Barcelona team with corresponding passes
def get2dMatrix(passesDataframe, senderColumnName, RecipientColumnName):
  arr = np.arange(225).reshape(15,15)
  i = 0
  j = 0
  passCount = 0
  for i in range(15):
    for j in range(15):
      passCount = 0
      for index, row in passesDataframe.iterrows():
        if(row[senderColumnName] == i+1 and  row[RecipientColumnName] == j+1):
          passCount += 1
        arr[i][j] = passCount    
      j += 1
    i += 1 
  return arr

In [ ]:
def getTeamMatrix(teamId, teamMatchId):
  
  # Getting Successful passes of specific team in specific match
  successfulPasses = eventsWorld_CupTags[(eventsWorld_CupTags['teamId'] == teamId) & (eventsWorld_CupTags['matchId'] == teamMatchId) & (eventsWorld_CupTags['eventId'] == 8) & (eventsWorld_CupTags['_id'] == 1801)]
  successfulPasses.reset_index(level=0, inplace=True)

  # Adding Sender/Recipient positions
  successfulPasses2 = successfulPasses['positions'].sum()
  del successfulPasses2[1::2]
  x2 = pd.Series(successfulPasses2)
  successfulPassesSender = pd.concat([successfulPasses, pd.DataFrame(x2)], axis=1)
  successfulPassesSender.rename(columns = {0:'Sender'}, inplace = True)

  successfulPasses1 = successfulPassesSender['positions'].sum()
  del successfulPasses1[0::2]
  x = pd.Series(successfulPasses1)
  successfulPassesSenderRecipient = pd.concat([successfulPassesSender, pd.DataFrame(x)], axis=1)
  successfulPassesSenderRecipient.rename(columns = {0:'Recepient'}, inplace = True) 

  # Adding Sender/Recipient zones

  zoneSeriesSender = checkZones2(successfulPassesSenderRecipient, 'Sender')
  zoneDataframeSender = pd.DataFrame(zoneSeriesSender)
  zoneDataframeSender.rename(columns = {0:'SenderZone'}, inplace = True)

  zoneSeriesRecipient = checkZones2(successfulPassesSenderRecipient, 'Recepient')
  zoneDataframeRecipient = pd.DataFrame(zoneSeriesRecipient)
  zoneDataframeRecipient.rename(columns = {0:'RecipientZone'}, inplace = True)

  successfulPassesSenderRecipientZ = pd.concat([successfulPassesSenderRecipient, zoneDataframeSender], axis=1)
  successfulPassesSenderRecipientZone = pd.concat([successfulPassesSenderRecipientZ, zoneDataframeRecipient], axis=1)
  
  # returning the 2D passes Matrix
  return get2dMatrix(successfulPassesSenderRecipientZone, 'SenderZone', 'RecipientZone').flatten()

In [ ]:
def getTeamMatches(teamId):
  teamMatchesIds = np.array([])
  eventsWorld_CupTeam = eventsWorld_Cup[eventsWorld_Cup['teamId'] == teamId]
  teamMatchesIds = eventsWorld_CupTeam.matchId.unique()
  return teamMatchesIds

In [ ]:
def getTeamMatrices(teamId):
  teamMatches = getTeamMatches(teamId)
  teamMatrices = np.zeros((len(teamMatches), 225))
  for i in range(len(teamMatches)):
    teamMatrices[i] = getTeamMatrix(teamId, teamMatches[i])
  return teamMatrices

In [ ]:
australiaMatrices = np.zeros((38, 225))
australiaMatrices = getTeamMatrices(8493)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
australiaMatrices

array([[15.,  7.,  8.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  4.,  5.,  4.,  8.,  0.,  4.,  2.,  2.,  1.,  3.,  0.,
         0.,  0.,  0.,  0.,  4.,  5.,  7.,  6.,  6.,  4.,  1.,  1.,  2.,
         2.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  4.,  2.,  6.,  1.,  1.,
         0.,  5.,  6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,  3.,
         0.,  0.,  0.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,  5.,  4.,  0.,
         0.,  0., 22., 14.,  2.,  0.,  0.,  3.,  2.,  1.,  0.,  0.,  1.,
         1.,  2.,  1.,  0., 12.,  3.,  2.,  3.,  1.,  0.,  2.,  0.,  0.,
         2.,  0.,  1.,  2.,  1.,  0.,  1.,  4.,  2.,  1.,  2.,  0.,  0.,
         1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  3.,  1.,  7.,
         0.,  0.,  0.,  2.,  2.,  0.,  0.,  4.,  2.,  0.,  0.,  1.,  2.,
         6., 19.,  1.,  0.,  1.,  1.,  5.,  0.,  0.,  0.,  0.,  0.,  2.,
         0.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.

In [ ]:
iranMatrices = np.zeros((38, 225))
iranMatrices = getTeamMatrices(10840)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
iranMatrices

array([[4., 2., 0., 0., 0., 4., 1., 0., 1., 0., 0., 0., 0., 0., 0., 2.,
        0., 0., 0., 0., 2., 1., 0., 0., 0., 0., 0., 0., 0., 0., 2., 3.,
        0., 0., 0., 1., 1., 0., 2., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
        0., 2., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 1.,
        9., 0., 0., 0., 2., 2., 0., 0., 1., 1., 0., 2., 0., 0., 0., 0.,
        4., 1., 0., 0., 1., 2., 1., 0., 0., 0., 0., 0., 1., 0., 0., 6.,
        3., 1., 2., 0., 1., 1., 2., 0., 0., 0., 0., 1., 0., 0., 0., 4.,
        3., 0., 1., 0., 0., 0., 0., 0., 0., 0., 2., 1., 0., 0., 1., 3.,
        2., 4., 1., 0., 0., 1., 2., 0., 1., 0., 0., 1., 0., 0., 0., 5.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 2., 1., 0., 0., 0.,
        3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        4., 2., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 

In [ ]:
koreaMatrices = np.zeros((38, 225))
koreaMatrices = getTeamMatrices(14855)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
koreaMatrices

array([[ 0.,  2.,  2.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  1.,  2.,  5.,  0.,  0.,  3.,  1.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  2.,  3.,  1.,  0.,  7.,  3.,  2.,  1.,  2.,
         0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  2.,  5.,  4.,  0.,  0.,
         2.,  1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  5.,  4.,
         0.,  0.,  0.,  5.,  2.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  3.,  6.,  5.,  0.,  0.,  3.,  2.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  4.,  8.,  7.,  4.,  1.,  1.,  1.,  0.,  0.,
         0.,  0.,  2.,  2.,  0.,  0.,  3.,  2.,  2., 10.,  2.,  1.,  1.,
         2.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  2.,  6.,  8.,  5.,
         0.,  0.,  0.,  3.,  5.,  0.,  0.,  0.,  0.,  1.,  0.,  3.,  0.,
         2., 12.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  8.,  2.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  3.,  1.

In [ ]:
japanMatrices = np.zeros((38, 225))
japanMatrices = getTeamMatrices(12913)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
japanMatrices

array([[ 4.,  2.,  5.,  1.,  0.,  2.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  2.,  3.,  1.,  1.,  0.,  2.,  7.,  2.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  3.,  5.,  5.,  3.,  1.,  6.,  1.,  7.,
         3.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  5.,  1.,  1.,  0.,  1.,
         4.,  3.,  4.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
         0.,  1.,  0.,  3.,  1.,  0.,  1.,  0.,  0.,  0.,  3.,  0.,  0.,
         1.,  0.,  7., 15.,  2.,  0.,  0.,  9.,  2.,  0.,  0.,  0.,  0.,
         1.,  2.,  2.,  0., 14., 14., 12.,  9.,  0.,  8.,  5.,  3.,  0.,
         1.,  0.,  0.,  5.,  1.,  2.,  4.,  9.,  6., 10.,  3.,  2.,  1.,
         0.,  3.,  2.,  0.,  1.,  1.,  4.,  0.,  1.,  7.,  8., 24., 14.,
         1.,  0.,  1.,  4.,  6.,  0.,  1.,  1.,  2.,  0.,  0.,  1.,  7.,
        11., 10.,  0.,  0.,  0.,  3., 13.,  1.,  0.,  0.,  0.,  0.,  5.,
         4.,  0.,  0.,  0., 14.,  6.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  4.,  0.,  0.,  0.,  6.,  1.,  2.

In [ ]:
saudiArabiaMatrices = np.zeros((38, 225))
saudiArabiaMatrices = getTeamMatrices(16521)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
saudiArabiaMatrices

array([[ 5.,  2.,  1.,  1.,  1.,  6.,  3.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  5.,  4.,  7.,  6.,  0.,  3.,  3.,  2.,  2.,  3.,  0.,
         0.,  1.,  0.,  0.,  2.,  6.,  1.,  2.,  1.,  1.,  7.,  1.,  1.,
         2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  6.,  5.,  5.,  1.,
         5.,  5.,  7.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  2.,  1.,
         0.,  0.,  1.,  5.,  5.,  0.,  0.,  0.,  0.,  0.,  2.,  5.,  1.,
         0.,  0., 11., 17.,  1.,  2.,  0.,  5.,  0.,  0.,  0.,  0.,  1.,
         6.,  4.,  5.,  0., 16., 10.,  6.,  3.,  3.,  0.,  3.,  2.,  0.,
         2.,  0.,  2.,  2.,  0.,  0.,  5.,  7.,  8.,  4.,  5.,  1.,  0.,
         0.,  1.,  2.,  0.,  2.,  1.,  3.,  1.,  1.,  3.,  8., 15., 12.,
         0.,  1.,  0.,  3.,  7.,  0.,  0.,  2.,  8.,  4.,  0.,  1.,  1.,
        17., 26.,  0.,  0.,  0.,  1.,  7.,  0.,  0.,  0.,  0.,  0.,  2.,
         2.,  1.,  0.,  0.,  7.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  1.

In [ ]:
tunisiaMatrices = np.zeros((38, 225))
tunisiaMatrices = getTeamMatrices(16276)

In [ ]:
tunisiaMatrices

array([[ 7.,  5.,  2.,  0.,  0.,  3.,  4.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  9.,  7.,  2.,  2.,  2.,  4.,  2.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  4.,  4.,  1.,  2.,  1.,  0.,  0.,  2.,  3.,
         4.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,  5.,  7.,  8.,  0.,  0.,
         0.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  7., 11.,
         0.,  0.,  0.,  3.,  6.,  0.,  0.,  0.,  0.,  0.,  2.,  7.,  0.,
         0.,  0.,  4.,  5.,  1.,  0.,  0.,  3.,  1.,  0.,  0.,  0.,  1.,
         1.,  1.,  1.,  1.,  8.,  2.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,
         0.,  0.,  2.,  1.,  0.,  0.,  1.,  2.,  1.,  2.,  3.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  2.,  3., 11.,  4.,
         0.,  0.,  0.,  3.,  1.,  0.,  1.,  1.,  2.,  3.,  0.,  0.,  1.,
         9., 10.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,
         2.,  0.,  0.,  0.,  5.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  2.

In [ ]:
senegalMatrices = np.zeros((38, 225))
senegalMatrices = getTeamMatrices(19314)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
senegalMatrices

array([[11.,  0.,  2.,  0.,  0.,  4.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  3.,  2.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  4.,  2.,  2.,  2.,  0.,  3.,  2.,
         2.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  2.,  3.,  3.,  1.,  2.,
         1.,  1.,  2.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  3.,
         0.,  0.,  0.,  1.,  6.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  2.,
         0.,  0., 11.,  8.,  3.,  1.,  0.,  5.,  3.,  0.,  0.,  0.,  0.,
         0.,  1.,  3.,  0.,  7.,  7.,  4.,  1.,  3.,  3.,  2.,  0.,  1.,
         0.,  0.,  0.,  1.,  1.,  0.,  2.,  5.,  3.,  1.,  5.,  2.,  1.,
         0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  3.,
         0.,  0.,  0.,  0.,  2.,  0.,  0.,  1.,  1.,  3.,  0.,  3.,  3.,
         3., 14.,  0.,  1.,  1.,  1.,  5.,  0.,  0.,  0.,  0.,  0.,  3.,
         0.,  1.,  0.,  2., 15.,  7.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
         0.,  2.,  1.,  0.,  2.,  0.,  2.,  2.,  4.

In [ ]:
moroccoMatrices = np.zeros((38, 225))
moroccoMatrices = getTeamMatrices(16216)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
moroccoMatrices

array([[ 1.,  2.,  1.,  0.,  0.,  1.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  4.,  3.,  4.,  0.,  2.,  4.,  1.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  2.,  2.,  3.,  3.,  1.,  1.,  2.,  3.,  3.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  3.,  2.,  0.,  2.,  2.,
         2.,  3.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  2.,  1.,
         0.,  1.,  1.,  1.,  3.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  1.,
         1.,  0., 21., 13.,  3.,  0.,  0.,  6.,  1.,  0.,  0.,  0.,  1.,
         2.,  1.,  1.,  1., 14., 14., 11., 11.,  0.,  3.,  6.,  1.,  0.,
         2.,  0.,  0.,  0.,  1.,  0.,  2., 16.,  4.,  5.,  2.,  0.,  0.,
         1.,  1.,  4.,  2.,  2.,  1.,  2.,  0.,  1.,  8.,  8., 16., 13.,
         1.,  0.,  1.,  4.,  2.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  4.,
        10.,  7.,  0.,  0.,  0.,  3.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  4.,  7.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  2.,  1.,  0.,  0.,  7.,  3.,  2.

In [ ]:
egyptMatrices = np.zeros((38, 225))
egyptMatrices = getTeamMatrices(16129)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
egyptMatrices

array([[ 7.,  3.,  1.,  0.,  0.,  7.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  5.,  4.,  2.,  4.,  0.,  5.,  1.,  3.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  2.,  3.,  4.,  2.,  0.,  1.,  4.,  4.,  1.,
         1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  3.,  3.,  2.,  0.,  0.,
         2.,  3.,  3.,  0.,  1.,  0.,  2.,  0.,  0.,  0.,  0.,  1.,  2.,
         0.,  0.,  0.,  1.,  4.,  0.,  0.,  0.,  0.,  0.,  1.,  6.,  2.,
         0.,  0., 24., 10.,  0.,  0.,  0.,  4.,  1.,  0.,  0.,  1.,  0.,
         0.,  1.,  0.,  0., 12., 11.,  5.,  1.,  0.,  3.,  1.,  0.,  2.,
         1.,  0.,  1.,  1.,  1.,  0.,  5.,  3.,  6.,  3.,  1.,  0.,  1.,
         0.,  0.,  4.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  3.,  3.,  8.,
         0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  2.,  1.,  0.,  0.,  1.,
         6., 13.,  0.,  0.,  0.,  1.,  4.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0., 19.,  6.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  2.,  1.,  0.,  0.,  3.,  6.,  1.

In [ ]:
nigeriaMatrices = np.zeros((38, 225))
nigeriaMatrices = getTeamMatrices(16823)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
nigeriaMatrices

array([[ 1.,  4.,  0.,  0.,  0.,  1.,  2.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  3.,  6.,  3.,  6.,  0.,  1.,  7.,  2.,  4.,  0.,  0.,
         0.,  0.,  0.,  0.,  2.,  7.,  1.,  3.,  1.,  1.,  3.,  4.,  3.,
         2.,  0.,  0.,  0.,  1.,  1.,  1.,  4.,  8.,  5.,  5.,  1.,  3.,
         1.,  1.,  3.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  5.,  3.,  4.,
         0.,  0.,  0.,  3.,  2.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
         0.,  0.,  2.,  4.,  0.,  0.,  0.,  1.,  2.,  0.,  0.,  0.,  2.,
         4.,  0.,  1.,  0.,  3., 12., 11.,  4.,  0.,  2.,  0.,  2.,  1.,
         0.,  0.,  0.,  1.,  2.,  0.,  1.,  6.,  8., 10.,  2.,  0.,  1.,
         1.,  3.,  1.,  0.,  2.,  1.,  2.,  0.,  1.,  5.,  9.,  8.,  7.,
         2.,  0.,  2.,  4.,  8.,  0.,  0.,  0.,  4.,  1.,  0.,  0.,  1.,
         6.,  7.,  0.,  0.,  0.,  1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  2.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  4.,  2.

In [ ]:
mexicoMatrices = np.zeros((38, 225))
mexicoMatrices = getTeamMatrices(15473)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
mexicoMatrices

array([[ 4.,  3.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  4.,  3.,  4.,  1.,  0.,  0.,  2.,  3.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  3.,  5.,  2.,  3.,  3.,  1.,  1.,  1.,
         3.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  3.,  3.,  1.,  0.,
         1.,  3.,  3.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
         0.,  0.,  1.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  2.,
         0.,  0.,  9.,  8.,  1.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,
         3.,  1.,  0.,  0.,  5.,  7.,  0.,  2.,  0.,  2.,  2.,  2.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  1.,  7.,  3.,  1.,  1.,  1.,  1.,
         1.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  2.,  0.,  2.,  6.,
         0.,  0.,  1.,  0.,  2.,  0.,  0.,  1.,  2.,  1.,  0.,  0.,  1.,
         2.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  3.,
         1.,  0.,  0.,  0.,  7.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  3.,  1.,  1.,  0.,  0.,  1., 12.,  5.

In [ ]:
costaRicaMatrices = np.zeros((38, 225))
costaRicaMatrices = getTeamMatrices(16871)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
costaRicaMatrices

array([[ 0.,  5.,  2.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  3.,  6.,  5.,  2.,  1.,  4.,  5.,  1.,  2.,  0.,  0.,
         0.,  1.,  0.,  1.,  2.,  6.,  3.,  3.,  0.,  3.,  2.,  2.,  4.,
         1.,  1.,  2.,  0.,  2.,  0.,  1.,  3.,  3.,  5.,  3.,  0.,  0.,
         3.,  2.,  3.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  1.,  4.,  3.,
         0.,  0.,  0.,  3.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  4.,  1.,
         0.,  0., 12.,  8.,  4.,  2.,  0.,  4.,  4.,  1.,  0.,  0.,  0.,
         2.,  4.,  2.,  0.,  8.,  2.,  8.,  0.,  0.,  2.,  6.,  3.,  2.,
         0.,  0.,  0.,  2.,  1.,  0.,  4.,  8.,  6.,  7.,  3.,  1.,  1.,
         1.,  2.,  1.,  0.,  0.,  2.,  1.,  3.,  1.,  3.,  4.,  0.,  9.,
         1.,  1.,  2.,  0.,  3.,  0.,  0.,  2.,  2.,  0.,  0.,  0.,  3.,
         5.,  5.,  0.,  0.,  1.,  4.,  4.,  0.,  0.,  0.,  0.,  0.,  2.,
         0.,  1.,  0.,  0.,  9.,  5.,  2.,  1.,  1.,  0.,  0.,  0.,  0.,
         0.,  2.,  2.,  2.,  0.,  0.,  3.,  2.,  2.

In [ ]:
panamaMatrices = np.zeros((38, 225))
panamaMatrices = getTeamMatrices(17929)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
panamaMatrices

array([[ 3.,  0.,  0.,  1.,  0.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  2.,  1.,  4.,  1.,  0.,  4.,  2.,  0.,  0.,  1.,  0.,
         1.,  1.,  0.,  0.,  0.,  2.,  4.,  0.,  1.,  2.,  0.,  0.,  1.,
         1.,  1.,  0.,  0.,  0.,  0.,  0.,  2.,  3.,  1.,  0.,  1.,  1.,
         1.,  4.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  7.,
         0.,  0.,  1.,  1.,  3.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  0.,
         1.,  0., 17.,  8.,  0.,  1.,  0.,  6.,  2.,  1.,  0.,  0.,  0.,
         2.,  1.,  2.,  0.,  8.,  6.,  2.,  1.,  1.,  0.,  2.,  0.,  1.,
         1.,  0.,  2.,  0.,  1.,  0.,  4.,  0.,  6.,  4.,  4.,  0.,  0.,
         3.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,  3.,  6.,  3.,  6.,
         2.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  2.,  1.,  0.,  0.,  0.,
         3., 21.,  1.,  1.,  0.,  1., 10.,  0.,  0.,  0.,  0.,  0.,  2.,
         1.,  0.,  0.,  0.,  6.,  4.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  2.,  1.,  3.

In [ ]:
brazilMatrices = np.zeros((38, 225))
brazilMatrices = getTeamMatrices(6380)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
brazilMatrices

array([[ 5.,  3.,  0., ...,  0.,  4.,  8.],
       [ 7.,  3.,  3., ...,  4.,  5.,  8.],
       [ 7.,  1.,  2., ...,  2.,  6.,  8.],
       [ 2.,  4.,  0., ...,  0.,  2., 15.],
       [ 4.,  3.,  2., ...,  1.,  6.,  8.]])

In [ ]:
argentinaMatrices = np.zeros((38, 225))
argentinaMatrices = getTeamMatrices(12274)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
argentinaMatrices

array([[ 3.,  1.,  2.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  3.,  3.,  3.,  1.,  0.,  0.,  4.,  4.,  2.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  5.,  6.,  3.,  3.,  0.,  3.,  2.,  1.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  5.,  2.,  5.,  0.,  2.,
         4.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  6.,  4.,
         0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
         0.,  0.,  7., 25.,  4.,  1.,  0.,  5.,  2.,  0.,  0.,  0.,  0.,
         1.,  0.,  1.,  0., 15., 23., 27., 11.,  1., 10., 15.,  8.,  3.,
         3.,  0.,  0.,  1.,  0.,  0.,  9., 17., 10., 19.,  6.,  4.,  4.,
         1.,  5.,  2.,  0.,  1.,  0.,  0.,  1.,  1., 14., 14., 15.,  9.,
         0.,  2.,  1., 13.,  5.,  0.,  1.,  1.,  1.,  0.,  0.,  3.,  2.,
        15.,  5.,  0.,  0.,  1.,  3.,  6.,  0.,  0.,  0.,  0.,  0.,  4.,
         9.,  0.,  1.,  0., 11., 18.,  1.,  2.,  1.,  0.,  0.,  0.,  0.,
         0.,  2.,  8.,  2.,  1.,  1., 15., 28., 11.

In [ ]:
peruMatrices = np.zeros((38, 225))
peruMatrices = getTeamMatrices(15594)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
peruMatrices

array([[ 6.,  2.,  2.,  0.,  0., 11.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  4.,  1.,  1.,  0.,  0.,  3.,  1.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  2.,  3.,  4.,  3.,  3.,  2.,  3.,  3.,
         2.,  0.,  1.,  0.,  1.,  0.,  0.,  2.,  2.,  2.,  1.,  0.,  2.,
         1.,  4.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  3.,  4.,
         0.,  0.,  0.,  3.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  2.,
         1.,  0., 19.,  1.,  1.,  0.,  2.,  2.,  1.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  5.,  2.,  5.,  1.,  0.,  0.,  0.,  0.,  1.,
         0.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  6.,  2.,  3.,  0.,  2.,
         2.,  2.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  2.,  2.,  1.,
         2.,  0.,  1.,  6.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         4.,  8.,  1.,  1.,  0.,  1.,  4.,  0.,  0.,  0.,  0.,  0.,  2.,
         2.,  0.,  0.,  0., 12.,  4.,  2.,  3.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  7., 11.,  4.

In [ ]:
uruguayMatrices = np.zeros((38, 225))
uruguayMatrices = getTeamMatrices(15670)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
uruguayMatrices

array([[ 2.,  1.,  1., ...,  0.,  8., 14.],
       [ 3.,  1.,  1., ...,  1.,  3.,  4.],
       [ 4.,  1.,  0., ...,  1.,  5.,  5.],
       [ 0.,  3.,  1., ...,  3.,  1., 11.],
       [ 0.,  1.,  0., ...,  3.,  6., 11.]])

In [ ]:
colombiaMatrices = np.zeros((38, 225))
colombiaMatrices = getTeamMatrices(12430)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
colombiaMatrices

array([[ 2.,  4.,  1.,  0.,  0.,  6.,  7.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  7.,  3.,  2.,  3.,  0.,  6.,  4.,  3.,  0.,  2.,  0.,
         0.,  0.,  0.,  0.,  2.,  1.,  1.,  1.,  1.,  0.,  2.,  0.,  2.,
         2.,  0.,  0.,  0.,  0.,  0.,  2.,  4.,  4.,  1.,  2.,  0.,  1.,
         3.,  6.,  3.,  0.,  0.,  0.,  2.,  0.,  0.,  1.,  0.,  6.,  5.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  4.,  1.,
         0.,  0.,  8.,  8.,  1.,  0.,  0.,  7.,  3.,  0.,  1.,  0.,  2.,
         2.,  0.,  1.,  0.,  9.,  7.,  6.,  0.,  1.,  3.,  1.,  0.,  0.,
         0.,  0.,  0.,  1.,  3.,  0.,  3.,  4.,  3.,  4.,  2.,  1.,  0.,
         1.,  3.,  0.,  0.,  2.,  1.,  1.,  0.,  0.,  1.,  2.,  4.,  2.,
         1.,  0.,  1.,  6.,  6.,  0.,  1.,  0.,  4.,  0.,  0.,  0.,  2.,
         3.,  7.,  0.,  0.,  1.,  2.,  3.,  0.,  0.,  0.,  0.,  0.,  2.,
         1.,  0.,  0.,  0.,  5.,  1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  2.,  0.,  1.,  0.,  2.,  0.,  1.

In [ ]:
belgiumMatrices = np.zeros((38, 225))
belgiumMatrices = getTeamMatrices(5629)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
belgiumMatrices

array([[ 0.,  3.,  1., ...,  2.,  4., 11.],
       [ 1.,  6.,  0., ...,  3.,  3.,  9.],
       [ 0.,  6.,  0., ...,  2.,  3.,  1.],
       ...,
       [ 1.,  5.,  2., ...,  1.,  1.,  0.],
       [ 0.,  3.,  1., ...,  2.,  8., 11.],
       [ 3.,  3.,  0., ...,  0.,  4.,  2.]])

In [ ]:
denmarkMatrices = np.zeros((38, 225))
denmarkMatrices = getTeamMatrices(7712)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
denmarkMatrices

array([[ 7.,  1.,  0.,  0.,  0.,  1.,  2.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  4.,  6.,  2.,  1.,  0.,  1.,  2.,  0.,  1.,  0.,  0.,
         1.,  0.,  0.,  0.,  1.,  2.,  3.,  0.,  2.,  0.,  2.,  2.,  6.,
         1.,  0.,  0.,  0.,  1.,  2.,  0.,  0.,  1.,  2.,  0.,  0.,  0.,
         0.,  1.,  2.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  2.,  1.,  0.,
         0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  2.,  1.,  0.,  0.,
         0.,  0.,  2.,  7.,  1.,  0.,  0.,  1.,  2.,  0.,  0.,  0.,  0.,
         0.,  3.,  0.,  0.,  5.,  9.,  6.,  6.,  0.,  2.,  2.,  0.,  1.,
         1.,  0.,  0.,  3.,  0.,  0.,  2.,  8.,  4.,  6.,  4.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  2.,  1.,  1.,  0.,  4.,  9.,  6.,  9.,
         2.,  0.,  2.,  2.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,
        11., 11.,  1.,  0.,  0.,  3.,  3.,  0.,  0.,  0.,  0.,  0.,  3.,
         0.,  0.,  0.,  0.,  9.,  5.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  4.,  8.,  2.

In [ ]:
englandMatrices = np.zeros((38, 225))
englandMatrices = getTeamMatrices(2413)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
englandMatrices

array([[ 2.,  5.,  1., ...,  0.,  8.,  7.],
       [ 2.,  2.,  2., ...,  0.,  3.,  4.],
       [ 1.,  8.,  3., ...,  1.,  2.,  2.],
       ...,
       [ 6.,  3.,  1., ...,  0., 10., 13.],
       [ 8.,  1.,  2., ...,  1.,  5.,  8.],
       [ 1.,  3.,  1., ...,  0., 13., 14.]])

In [ ]:
franceMatrices = np.zeros((38, 225))
franceMatrices = getTeamMatrices(4418)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
franceMatrices

array([[ 2.,  1.,  2., ...,  1.,  5., 20.],
       [ 3.,  4.,  1., ...,  0.,  4.,  1.],
       [ 5.,  2.,  3., ...,  4., 13., 25.],
       ...,
       [ 5.,  3.,  2., ...,  2.,  4., 25.],
       [ 4.,  4.,  1., ...,  3.,  2.,  9.],
       [12.,  2.,  0., ...,  0.,  0.,  5.]])

In [ ]:
croatiaMatrices = np.zeros((38, 225))
croatiaMatrices = getTeamMatrices(9598)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
croatiaMatrices

array([[ 3.,  1.,  3., ...,  0.,  2.,  8.],
       [ 4.,  5.,  2., ...,  2.,  0.,  4.],
       [ 5.,  2.,  1., ...,  3.,  1.,  8.],
       ...,
       [ 3.,  3.,  5., ...,  4.,  6., 14.],
       [ 6.,  1.,  0., ...,  4.,  5., 20.],
       [ 8.,  1.,  0., ...,  2.,  6.,  9.]])

In [ ]:
germanyMatrices = np.zeros((38, 225))
germanyMatrices = getTeamMatrices(3148)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
germanyMatrices

array([[ 2.,  2.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  2.,  3.,  3.,  3.,  1.,  1.,  2.,  1.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  1.,  2.,  2.,  0.,  1.,  5.,  1.,  1.,
         2.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  3.,  4.,  2.,  0.,  0.,
         1.,  4.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  3.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
         0.,  0.,  6.,  6.,  3.,  0.,  0.,  4.,  0.,  2.,  0.,  0.,  0.,
         1.,  1.,  1.,  0.,  4.,  7.,  4.,  6.,  2.,  4.,  1.,  0.,  1.,
         1.,  0.,  0.,  0.,  0.,  0.,  2.,  3.,  3.,  9.,  6.,  4.,  0.,
         3.,  1.,  4.,  0.,  0.,  0.,  6.,  0.,  1.,  2.,  9., 20., 21.,
         3.,  1.,  5.,  8., 13.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  2.,
        18., 16.,  1.,  0.,  2.,  3., 14.,  0.,  0.,  0.,  0.,  0.,  4.,
         2.,  1.,  0.,  1., 16., 11.,  4.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0.,  3.,  8.,  3.

In [ ]:
icelandMatrices = np.zeros((38, 225))
icelandMatrices = getTeamMatrices(7839)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
icelandMatrices

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  3.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
         2.,  0.,  0.,  0.,  1.,  2.,  5.,  5.,  2.,  0.,  1.,  2.,  0.,
         1.,  0.,  2.,  0.,  1.,  0.,  0.,  0.,  2.,  4.,  2.,  0.,  1.,
         0.,  3.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  1.,  2.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  2.,
         0.,  0.,  2.,  3.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
         0.,  2.,  0.,  0.,  5.,  2.,  1.,  0.,  0.,  4.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  2.,  4.,  1.,  1.,  0.,  2.,  2.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  4.,  2.,  0.,
         0.,  0.,  2.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  4.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  1.,  1.,  5.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  1.,  3.,  1.

In [ ]:
polandMatrices = np.zeros((38, 225))
polandMatrices = getTeamMatrices(13869)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
polandMatrices

array([[ 1.,  2.,  5.,  0.,  0.,  2.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  2.,  6.,  7.,  5.,  0.,  6.,  8.,  8.,  3.,  0.,  0.,
         0.,  0.,  0.,  0.,  2., 10.,  1.,  3.,  1.,  2., 11.,  5.,  3.,
         3.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,  7.,  3.,  5.,  0.,  3.,
         6.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  4.,  2.,
         0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  2.,  6.,  1.,
         0.,  0.,  9., 15.,  7.,  0.,  0.,  3.,  2.,  1.,  0.,  1.,  0.,
         4.,  1.,  3.,  0., 16.,  8., 18.,  7.,  2.,  2.,  7.,  1.,  0.,
         2.,  0.,  3.,  1.,  3.,  0., 10., 13.,  6., 11.,  5.,  3.,  4.,
         2.,  0.,  1.,  0.,  0.,  4.,  5.,  1.,  1.,  8.,  7.,  6.,  4.,
         1.,  1.,  0.,  4.,  2.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  2.,
        13.,  3.,  0.,  0.,  0.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,
         3.,  0.,  0.,  0.,  3.,  4.,  1.,  2.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0.,  4.,  5.,  4.

In [ ]:
portugalMatrices = np.zeros((38, 225))
portugalMatrices = getTeamMatrices(9905)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
portugalMatrices

array([[ 1.,  1.,  3.,  0.,  0.,  4.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  2.,  4.,  1.,  0.,  7.,  2.,  1.,  1.,  0.,  0.,
         0.,  1.,  0.,  0.,  3.,  2.,  3.,  1.,  1.,  3.,  0.,  3.,  1.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  2.,  3.,  1.,  0.,
         1.,  3.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  3.,  1.,
         1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  4.,  0.,
         0.,  0., 10.,  9.,  3.,  0.,  0., 11.,  0.,  1.,  1.,  0.,  2.,
         1.,  0.,  0.,  0.,  9., 10.,  6.,  5.,  0.,  6.,  1.,  1.,  1.,
         0.,  1.,  2.,  0.,  0.,  0.,  6.,  4., 11.,  4.,  5.,  1.,  0.,
         2.,  1.,  1.,  0.,  0.,  0.,  2.,  1.,  0.,  3.,  8.,  3.,  4.,
         3.,  2.,  1.,  1.,  2.,  0.,  0.,  1.,  2.,  1.,  0.,  0.,  0.,
         2.,  2.,  1.,  2.,  1.,  3.,  2.,  0.,  0.,  0.,  0.,  0.,  8.,
         3.,  0.,  0.,  0.,  4.,  5.,  1.,  0.,  2.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  2.,  0.,  0.,  3.,  2.,  3.

In [ ]:
russiaMatrices = np.zeros((38, 225))
russiaMatrices = getTeamMatrices(14358)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
russiaMatrices

array([[ 2.,  0.,  1., ...,  0.,  2., 11.],
       [ 2.,  0.,  0., ...,  1.,  9., 18.],
       [ 5.,  1.,  1., ...,  1.,  5., 19.],
       [17.,  5.,  0., ...,  0.,  0., 10.],
       [ 2.,  1.,  2., ...,  3.,  5.,  8.]])

In [ ]:
serbiaMatrices = np.zeros((38, 225))
serbiaMatrices = getTeamMatrices(17322)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
serbiaMatrices

array([[ 0.,  1.,  1.,  0.,  0.,  1.,  2.,  0.,  1.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.,  0.,  2.,  2.,  1.,  4.,  3.,  0.,  3.,  0.,  0.,
         1.,  0.,  0.,  0.,  1.,  6.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  7.,  1.,  2.,  1.,  0.,  1.,
         1.,  4.,  4.,  1.,  0.,  0.,  3.,  0.,  0.,  0.,  0.,  1.,  2.,
         0.,  0.,  0.,  1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  7., 10.,  1.,  4.,  0.,  2.,  3.,  1.,  1.,  0.,  1.,
         3.,  0.,  4.,  0.,  9.,  9.,  2.,  9.,  1.,  2.,  4.,  2.,  1.,
         0.,  0.,  2.,  0.,  1.,  0.,  2.,  5.,  1.,  3.,  3.,  0.,  0.,
         1.,  0.,  1.,  0.,  1.,  0.,  3.,  0.,  1.,  4.,  5.,  1., 18.,
         2.,  1.,  5.,  3.,  2.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,  1.,
        12., 10.,  0.,  0.,  1.,  1.,  9.,  0.,  0.,  0.,  0.,  0.,  5.,
         1.,  0.,  0.,  0.,  1.,  2.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  2.,  0.,  0.,  0.,  4.,  4.

In [ ]:
spainMatrices = np.zeros((38, 225))
spainMatrices = getTeamMatrices(1598)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
spainMatrices

array([[ 6.,  1.,  2.,  2.,  0.,  3.,  3.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  3.,  3.,  2.,  2.,  1.,  8.,  5.,  0.,  2.,  3.,  0.,
         0.,  0.,  0.,  0.,  1.,  4.,  2.,  0.,  2.,  2.,  5.,  2.,  1.,
         3.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  3.,  4.,  1.,  0.,  1.,
         3.,  5.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  5.,  2.,  3.,
         0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  3.,  5.,  0.,
         1.,  0., 51., 37.,  5.,  1.,  1.,  8.,  0.,  0.,  0.,  0.,  0.,
         4.,  1.,  4.,  0., 35., 36., 14.,  9.,  3.,  7.,  6.,  0.,  0.,
         3.,  0.,  1.,  1.,  0.,  0.,  3., 19., 12.,  4.,  4.,  4.,  0.,
         1.,  2.,  1.,  0.,  2.,  2.,  4.,  1.,  1., 14.,  9., 17., 14.,
         1.,  1.,  1.,  7.,  7.,  0.,  0.,  0.,  5.,  3.,  0.,  0.,  7.,
        21., 35.,  1.,  0.,  0.,  6.,  8.,  0.,  0.,  0.,  0.,  0., 11.,
         1.,  0.,  0.,  1., 22., 10.,  0.,  3.,  0.,  0.,  0.,  0.,  1.,
         0.,  1.,  0.,  1.,  0.,  0., 10.,  4.,  3.

In [ ]:
swedenMatrices = np.zeros((38, 225))
swedenMatrices = getTeamMatrices(7047)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
swedenMatrices

array([[1., 2., 0., ..., 2., 5., 4.],
       [0., 2., 0., ..., 1., 3., 5.],
       [2., 1., 3., ..., 1., 2., 3.],
       [0., 1., 0., ..., 0., 2., 2.],
       [4., 0., 2., ..., 1., 4., 7.]])

In [ ]:
switzerlandMatrices = np.zeros((38, 225))
switzerlandMatrices = getTeamMatrices(6697)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
switzerlandMatrices

array([[ 6.,  3.,  2.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  1.,  4.,  6.,  1.,  0.,  1.,  2.,  4.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  2.,  6.,  3.,  1.,  2.,  4.,  5.,  5.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  2.,  0.,  2.,  0.,  3.,
         1.,  1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  4.,  6.,
         0.,  0.,  0.,  1.,  4.,  0.,  0.,  0.,  0.,  1.,  1.,  2.,  2.,
         0.,  0.,  7.,  9.,  6.,  0.,  0.,  6.,  3.,  0.,  0.,  0.,  0.,
         1.,  3.,  0.,  0., 11., 10., 14.,  1.,  0.,  4.,  3.,  2.,  0.,
         0.,  0.,  3.,  3.,  1.,  0.,  8., 13., 13.,  7.,  5.,  2.,  0.,
         1.,  0.,  0.,  0.,  1.,  0.,  2.,  2.,  0.,  2.,  1.,  4.,  6.,
         1.,  0.,  1.,  2.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  1.,
         5., 13.,  0.,  0.,  0.,  4.,  4.,  0.,  0.,  0.,  0.,  0.,  2.,
         3.,  0.,  0.,  0.,  7.,  6.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  2.,  0.,  0.,  0.,  0.,  0.,  3.,  3.

In [ ]:
Matrices = np.zeros((38, 225))
Matrices = getTeamMatrices()

In [ ]:
Matrices = np.zeros((38, 225))
Matrices = getTeamMatrices()

In [ ]:
from google.colab import files
import pickle
with open('wc.pickle', 'wb') as f:
  pickle.dump([australiaMatrices, iranMatrices, koreaMatrices, japanMatrices, saudiArabiaMatrices, tunisiaMatrices, senegalMatrices, moroccoMatrices, egyptMatrices, nigeriaMatrices, mexicoMatrices, costaRicaMatrices, panamaMatrices, brazilMatrices, argentinaMatrices, peruMatrices, uruguayMatrices, colombiaMatrices, belgiumMatrices, croatiaMatrices, denmarkMatrices, englandMatrices, franceMatrices, germanyMatrices, icelandMatrices, polandMatrices, portugalMatrices, russiaMatrices, serbiaMatrices, spainMatrices, swedenMatrices, switzerlandMatrices], f)
files.download('wc.pickle')  